In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
from collections import defaultdict
import datetime

import dateutil
from dateutil.relativedelta import relativedelta

In [ ]:
alias_map = {}
clean = lambda x: ''.join(x.split()).lower()

In [ ]:
with open('arena_matches.pkl','rb') as fp: #cs_matches
    results = pickle.load(fp)

In [ ]:

data = []
for k in results:
    basics = [k] + list(results[k][:3]) +[results[k][-1]]
    for m in results[k][3]:
        new_elem = basics + list(m)
        if new_elem[5] == '':
            new_elem[5] = new_elem[3]
        if new_elem[5] == '':
            continue
        t1 = clean(new_elem[6])
        t2 = clean(new_elem[7])
        
        
        new_elem[6] = alias_map.get(t1,t1)
        new_elem[7] = alias_map.get(t2,t2)
        

        data.append(new_elem)

In [ ]:
new_elem,len(data)

In [ ]:
df = pd.DataFrame(data,columns=['venue','tier','type','vtime','game','mtime','t1','t2','s1','s2','rnds'])
df['mtime'] = df['mtime'].map(lambda x: dateutil.parser.parse(x.replace('??','01')) if type(x) == str else x)
df['mtime'] = pd.to_datetime(df['mtime'],utc=True)

df['t1'] = df['t1'].map(lambda x: x.lower())
df['t2'] = df['t2'].map(lambda x: x.lower())


df = df[df['t1'] != 'bye']
df = df[df['t2'] != 'bye']
df = df[df['t1'] != '']
df = df[df['t2'] != '']
df = df[~df.t1.map(lambda x: 'bye' in x)]
df = df[~df.t2.map(lambda x: 'bye' in x)]
df = df[~df.t1.map(lambda x: 'tbd' == x)]
df = df[~df.t2.map(lambda x: 'tbd' == x)]
df = df[~df.t1.map(lambda x: 'bracketmatch' in x)]
df = df[~df.t2.map(lambda x: 'bracketmatch' in x)]

subs = [('dj:)','dj'),('destrukt','dkt'),('razerj','jibo'),('a1r-r41d','a1r_r41d'),('cooler','cooller')]
subs += [('toxic','toxjq'),('tox','toxjq'),('dragoax','dragaox'),('razerj','jibo'),('razerjibo','jibo'),('fox','fojji')]
subs += [('matr0xpl','matr0xpl'),('toxic(americanplayer){{!}}toxic','toxic_us'),('solll_','solll')]
subs += [('inz_', 'inz'), ('j_skey', 'jskey'), ('memento_mori', 'mementomori'), ('pie_hero', 'piehero'), ('septic_flesh', 'septicflesh'), ('sib_', 'sib'), ('st_germain', 'stgermain'), ('zanes_', 'zanes')]
subs += [('brute-', 'brute'),('dr.doom', 'drdoom'), ('ms.x', 'msx'),('1grayfoxz', 'grayfoxz'),('astz!', 'astz')]
subs += [('buggunsjoey', 'biggunsjoey')]
subs += [('bootss', 'boots'),('bootsss', 'boots'),('ch33rra', 'ch3rra')]
subs += [('zblazin', 'blazin'),('troolzy', 'troolz'),('vedmedik', 'vedemedik'),('vigur', 'vigurr'),('spawn2k', 'spawn2k4'),('spawn2k4', 'spawn2k'),('stenx', 'strenx'),('scoob', 'scooby'),('richochet', 'ricochet'),('sassmaster2', 'sassmaster'),('phosgenee', 'phosgene'),('overflux1', 'overflux'),('nonserium', 'nonservium'),('naaani', 'naani'),('maggotkil', 'maggotkill'),('k11lsen', 'k1llsen'),('k1lsen', 'k1llsen'),('lakermann', 'lakerman'),('d-pecka', 'pecka'),('dazatorr', 'dazator'),('defense', 'defense1'),('headbang3rr', 'headbang3r'),('infinte', 'infinite'),('karwik', 'karwick')]
subs += [('?joker', 'joker')]
for old, new in subs:
    df.loc[df['t1'] == old,'t1'] = new
    df.loc[df['t2'] == old,'t2'] = new

df.loc[df['game'] == 'db','game'] = 'dbt'

df.loc[df['game'].map(lambda x: x in['ut99','ut2k3','ut2k4']),'game'] = 'unreal'

df = df[df.game != 'dbt']
df = df[df.game != 'reflex']

df.loc[df['tier'] != '1','tier'] = 'O'

df.loc[df.venue.map(lambda x: 'XSReality' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'MediaMarkt' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'Invitational' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'LAN Arena' in x),'tier'] = 'O'
df.loc[df.venue.map(lambda x: 'Atomic' in x),'tier'] = 'O'

#df = df[df.tier.isin(['A','S'])]
df['type'] = df['type'].map(lambda x: 'Online' if 'Offline' != x else 'Offline')
print(df.shape)

In [ ]:
df.game.value_counts()

In [ ]:
num_events = dict(df.tier.value_counts())
num_type = dict(df.type.value_counts())
N = df.shape[0]
num_type

score_mul = {  }
for t in num_type:
    for e in num_events:
        score_mul[(e,t)] = (N/num_type[t]) * (N/num_events[e])
        if t == 'Offline':
            score_mul[(e,t)]  *= 5
        if e == '1':
            score_mul[(e,t)]  *= 2
score_mul


In [ ]:
df = df.sort_values('mtime',0,kind='mergesort')

In [ ]:
import numpy as np
import datetime


sigmoid = lambda x: 1/(1+10**(-x/400)) 

def sigmoid(x):
    "Numerically-stable sigmoid function."
    if x >= 0:
        z = 10**(-x/400.0)
        return 1 / (1 + z)
    else:
        z = 10**(x/400.0)
        return z / (1 + z)
def benchmark_func(x):

    ratings = defaultdict(lambda: 1500)
    ratings_game = defaultdict(lambda: 1500)

    prev_update = {}
    x = (np.array(x).flatten())
    

    #print(blend,amount)

    x = abs(x)
    per_ratings = {
        ('O','Online'): x[0],
        ('O','Offline'): x[1],
        ('1','Online'): x[2],
        ('1','Offline'): x[3],
    }
    per_ratings_const = {
        ('O','Online'): x[4],
        ('O','Offline'): x[5],
        ('1','Online'): x[6],
        ('1','Offline'): x[7],
    }
    power = x[8]
    power2 = x[9]
    cMean = 100*x[10]
    cVar =  x[11]
    blend= np.exp(-x[12])
    amount= np.exp(-x[13])
    
    fix_hack = np.exp(-x[14])
    fix_hack2 = np.exp(-x[15])
    blend2 = np.exp(-x[16])
    blend3 = np.exp(-x[17])
    target_reset = np.exp(-x[18])
    #print(target_reset)
    score = 0.0

    correct = 0
    total_g = 0

    for row in df.itertuples():
        game = row[5]
        t1 = row[7] #+ '_' + row[5]
        t2 = row[8] #+ '_' + row[5]
        cTime = row[6].replace(tzinfo=None)
        time1 = prev_update.get(t1,cTime)
        time2 = prev_update.get(t2,cTime)

        total = row[9]+row[10]
        diff = abs(row[9]-row[10])
        if total < 1 or diff <1:
            prev_update[t1] = cTime
            prev_update[t2] = cTime
            continue
        diff /= total

        # get rating
        r1 = ratings[t1]
        r2 = ratings[t2]
        
        # time adjust
        td1 = float((cTime - time1).days)
        to1500_1 = float(sigmoid((td1-cMean)*cVar))*target_reset
        r1 = (to1500_1)*1500 + (1-to1500_1)*r1

        td2 = float((cTime - time2).days)
        to1500_2 = float(sigmoid((td2-cMean)*cVar))*target_reset
        r2 = (to1500_2)*1500 + (1-to1500_2)*r2

        # get per_game rating
        t1g = t1 + '_' + game
        t2g = t2 + '_' + game
        r1g = ratings_game[t1g]
        r2g = ratings_game[t2g]
        blend_x1 = blend2 if r1 > r1g else blend3
        blend_x2 = blend2 if r2 > r2g else blend3
        r1 = blend_x1*r1 + (1-blend_x1)*r1g
        r2 = blend_x2*r2 + (1-blend_x2)*r2g

        rdiff = abs(r1-r2)
        p1 = sigmoid(r1-r2)
        p2 = 1 - p1
        K = (diff**power) * (per_ratings[row[2:4]] ) + per_ratings_const[row[2:4]]
        K = K*blend + amount*rdiff*(1-blend)

        bot,top = sorted([row[9],row[10]])
        if top < 2 and top > 1:
            K *= fix_hack
        if top < 3 and top > 2:
            K *= fix_hack2
        #print(blend)
        #raise
        #if r1+r2 > 3400:

        #else:
        #    correct += 0.5
        #    total_g += 1
        sMul = score_mul[row[2:4]] * max(0.1,(r1+r2)/3000.0)**2
        #print(K,sMul)

        correct += int((row[9] > row[10]) == ( r1 > r2)) *sMul
        total_g += sMul

        if row[9] > row[10]:
            E = p2
            ratings[t1] = r1 + K * E**power2
            ratings[t2] = r2 - K * E**power2

            score += -np.log(max(p1,5e-2)) * sMul
        else:
            E = p1
            ratings[t1] = r1 - K * E**power2
            ratings[t2] = r2 + K * E**power2

            score += -np.log(max(p2,5e-2)) * sMul
        prev_update[t1] = cTime
        prev_update[t2] = cTime
        ratings_game[t1g] = ratings[t1]
        ratings_game[t2g] = ratings[t2]
    return score/max(1e-3,(correct)/total_g-0.5)**0.25#,ratings
initial_v = 2*np.ones(19).astype(np.float32)

initial_v = np.array([5.9678663326794110e+00, 1.5774535573337397e+02,
       7.9998985185720272e+01, 1.5301068037027565e+02,
       1.6144410032608093e+02, 9.5164564316202586e+01,
       9.8344679140760967e+00, 1.4372708126231532e+02,
       1.2110950773553764e+00, 1.3784564398888122e+00,
       4.1233581688270895e+01, 2.6016656390320172e+01,
       1.7597158111754247e+00, 1.5071212075843706e+01,
       6.1697980419537410e+00, 1.1114853479067094e-04,
       3.6811018877442131e-01, 5.6083643465419231e-01,1])
initial_v[10] = 7
initial_v[11] = 1.65
initial_v = np.array([5.9652665427524276e+00, 1.5652475356173554e+02,
       7.9708412792431417e+01, 1.5403795886634441e+02,
       1.5908686145618992e+02, 9.4955756441496902e+01,
       9.8535290482300653e+00, 1.4459492954875648e+02,
       1.2422358763007020e+00, 1.2969060690314214e+00,
       7.0120433227096166e+00, 1.6134028603747861e+00,
       1.7762535719086343e+00, 1.5042305471662427e+01,
       6.2085357431518879e+00, 7.1672059974945214e-07,
       3.7085077481778683e-01, 5.8464950390286585e-01,
       9.6715288119620679e-01])


s= benchmark_func(initial_v)
s,s<234127

In [ ]:
import cma #
es = cma.CMAEvolutionStrategy(initial_v, 0.001, {'tolx':1e-6,'tolfun':1e-6,})
es.optimize(benchmark_func,n_jobs=0)

In [ ]:
es.result_pretty()  #

In [ ]:
np.set_printoptions(suppress=False)
np.set_printoptions(precision=16)
abs(es.best.x)

In [ ]:
#print(benchmark_func(np.array([ 5, 25 , 2,  10, 31, 45, 1])))
#print(benchmark_func(np.array([ 2, 22 , 0.2,  9, 26, 42, 0.5])))
print(benchmark_func(initial_v))
print(benchmark_func(es.best.x))


In [ ]:
import GPy
import GPyOpt
bounds = [{'name': i, 'type': 'continuous', 'domain': (v[0],v[1])} for i,v in enumerate(zip(initial_v*0.99,initial_v*1.01))]
myProblem = GPyOpt.methods.BayesianOptimization(benchmark_func,bounds,
                                                model_type='GP',
                                                verbosity=True,
                                                verbose=True,
                                                verbose_model=True,
                                                acquisition_type='EI',
                                                exact_feval = True,
                                                normalize_Y=True, 
                batch_size=1, num_cores=4,
                verbosity_model=True, de_duplication=True)

myProblem.run_optimization(100000,60*45)
print(myProblem.fx_opt)


In [ ]:
benchmark_func(myProblem.x_opt)

In [ ]:
abs(myProblem.x_opt)

In [ ]:
# you might wonder why two versions of the same code
# this is a silly way for me to force myself to do code review for myself
from collections import defaultdict
ratings = defaultdict(lambda: 1500)
rt = defaultdict(list)
ratings_game = defaultdict(lambda: 1500)
prev_update = {}

per_ratings = {
    ('O','Online'): initial_v[0],
    ('O','Offline'): initial_v[1],
    ('1','Online'): initial_v[2],
    ('1','Offline'): initial_v[3],
}
per_ratings_const = {
    ('O','Online'): initial_v[4],
    ('O','Offline'): initial_v[5],
    ('1','Online'): initial_v[6],
    ('1','Offline'): initial_v[7],
}
power = initial_v[8]
power2 = initial_v[9]
cMean = 100*initial_v[10]
cVar =  initial_v[11]
blend= np.exp(-initial_v[12])
amount= np.exp(-initial_v[13])

fix_hack = np.exp(-initial_v[14])
fix_hack2 = np.exp(-initial_v[15])
blend2 = np.exp(-initial_v[16])
blend3 = np.exp(-initial_v[17])
target_reset = np.exp(-initial_v[18])

print(fix_hack)
score = 0.0

correct = 0
total_g = 0

for row in df.itertuples():
    game = row[5]
    t1 = row[7] #+ '_' + row[5]
    t2 = row[8] #+ '_' + row[5]
    cTime = row[6].replace(tzinfo=None)
    time1 = prev_update.get(t1,cTime)
    time2 = prev_update.get(t2,cTime)

    total = row[9]+row[10]
    diff = abs(row[9]-row[10])
    if total < 1 or diff <1:
        prev_update[t1] = cTime
        prev_update[t2] = cTime
        continue
    diff /= total

    # get rating
    r1 = ratings[t1]
    r2 = ratings[t2]
    td1 = float((cTime - time1).days)
    to1500_1 = float(sigmoid((td1-cMean)*cVar))*target_reset
    r1 = (to1500_1)*1500 + (1-to1500_1)*r1

    td2 = float((cTime - time2).days)
    to1500_2 = float(sigmoid((td2-cMean)*cVar))*target_reset
    r2 = (to1500_2)*1500 + (1-to1500_2)*r2
    
    if td1 > 300:
        rt[t1].append((cTime,np.nan,row[5]))
    if td2 > 300:
        rt[t2].append((cTime,np.nan,row[5]))

    # get per_game rating
    t1g = t1 + '_' + game
    t2g = t2 + '_' + game
    r1g = ratings_game[t1g]
    r2g = ratings_game[t2g]
    blend_x1 = blend2 if r1 > r1g else blend3
    blend_x2 = blend2 if r2 > r2g else blend3
    r1 = blend_x1*r1 + (1-blend_x1)*r1g
    r2 = blend_x2*r2 + (1-blend_x2)*r2g

    
    rdiff = abs(r1-r2)
    p1 = sigmoid(r1-r2)
    p2 = 1 - p1
    K = (diff**power) * (per_ratings[row[2:4]] ) + per_ratings_const[row[2:4]]
    K = K*blend + amount*rdiff*(1-blend)

    bot,top = sorted([row[9],row[10]])
    if top < 2 and top > 1:
        K *= fix_hack
    if top < 3 and top > 2:
        K *= fix_hack2
    #print(blend)
    #raise
    #if r1+r2 > 3400:

    #else:
    #    correct += 0.5
    #    total_g += 1
    sMul = score_mul[row[2:4]] * max(0.1,(r1+r2)/3000.0)**2
    #print(K,sMul)

    correct += int((row[9] > row[10]) == ( r1 > r2)) *sMul
    total_g += sMul

    if row[9] > row[10]:
        E = p2
        ratings[t1] = r1 + K * E**power2
        ratings[t2] = r2 - K * E**power2

        score += -np.log(max(p1,5e-2)) * sMul
    else:
        E = p1
        ratings[t1] = r1 - K * E**power2
        ratings[t2] = r2 + K * E**power2

        score += -np.log(max(p2,5e-2)) * sMul
    prev_update[t1] = cTime
    prev_update[t2] = cTime
    ratings_game[t1g] = ratings[t1]
    ratings_game[t2g] = ratings[t2]

    rt[t1].append((cTime,ratings[t1],row[5]))
    rt[t2].append((cTime,ratings[t2],row[5]))
score,100*correct/total_g,score*(1-correct/total_g),score/max(1e-3,(correct)/total_g-0.5)**0.25#,ratings

# 57841.09221530504
# per_ratings = {('A','Online'): 6.35,
#               ('S','Online'): 29.43,
#               ('A','Offline'): 8.56,
#               ('S','Offline'): 17.6}

#const = 14.86
#weeks = 45
#power = 2.24
# 17185 w/ 61.8 to get 6550 (no games)
# 16616 w/ 59.1 to get 6801 (games)
# 15940 w/ 56.14 to get 6991 (no games)
# 15986 w/ 53.72 to get 7397.8 (games)

# 16991 w/ 64.9 to get 5966 and 11414 (games)
# 17172 w/ 68.1 to get 5470 and 94649 (no games)357033

In [ ]:
blend2,blend3,target_reset

In [ ]:
#cMean,cVar = 1000,1
xn = np.linspace(0,365*5)
yn = [sigmoid((_-cMean)*cVar)*target_reset for _ in xn]
plt.plot(xn,yn)
cMean,cVar,initial_v[10],initial_v[11]


In [ ]:
sorted([(v,k) for k,v in ratings.items()],reverse=True)

In [ ]:
top_teams = []
for p,res in rt.items():
    for game in df.game.unique():
        if game == 'q3':
            v = [_ for _ in res if _[2] == game  and _[0].year < 2006 and np.isfinite(_[1])]
        else:
            v = [_ for _ in res if _[2] == game and np.isfinite(_[1])]

        if p =='fojji' and game == 'q3':
            pass#raise
        if len(v) > 0:
            #v = v[len(v)//3:]
            best_idx = np.argmax(np.array(v)[:,1])
            top_teams.append((v[best_idx][1:],p,v[best_idx][0]))
top_teams = sorted(top_teams,reverse=True)
#top_teams = sorted([(v[np.argmax(np.array(v)[:,1])][1:],k) for k,v in rt.items() if len(v) > 0],reverse=True)
top_teams

In [ ]:
df[((df.t1 == 'strenx') | (df.t2 == 'strenx')) &  (df.game == 'ql') &  (df.mtime.map(lambda x: x.year == 2011))]

In [ ]:
#df[df.venue == 'Quake Pro League/Season 1/Stage 1/Finals']
rt['strenx']

In [ ]:
color_dict = defaultdict(lambda: ('#ff00ff'))
color_dict['zero4'] = '#cc3311'
color_dict['fatal1ty'] = '#66CCEE'
color_dict['cooller'] = '#CCBB44'
color_dict['cypher'] = '#AA3377'
color_dict['rapha'] = '#4477AA'
color_dict['toxjq'] = '#228833'
color_dict['vo0'] = '#EE8866' 

color_dict['czm'] = '#6ab04c'
color_dict['z4muz'] = '#e58e26'
color_dict['lexer'] = '#6555e0'
color_dict['av3k'] = '#0077bb'
color_dict['dahang'] = '#663333'
color_dict['fojji'] = '#EE6677'
color_dict['ztrider'] = '#222255'
color_dict['gellehsak'] = '#bb5566'
color_dict['stermy'] = '#bbcc33'
color_dict['zyz'] = '#ddaa33'


#'#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE','#AA3377',

In [ ]:
import matplotlib.pyplot as plt
import datetime
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from collections import OrderedDict
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
for game in df.game.unique():#['q3','pk','q4','ql','qc']:
    plt.figure(figsize=(10,5),dpi=120)
    tt = [_ for _ in top_teams if game == _[0][1]]
    if game == 'qc':
        tt = [_ for _ in tt if _[1] != 'evil']
    for i in range(min(len(tt),6)):
        tm = tt[i][1]
        team_plot = np.array([_ for _ in rt[tm] if _[2] == game and np.isfinite(_[1])])
        c = color_dict[tm]
        plt.plot(team_plot[:,0],team_plot[:,1],label=tm,lw=5.5,alpha=0.72)

        team_plot = team_plot[min(5,len(team_plot)-1):]
        argmin = np.argmax(team_plot[:,1])
        peak = team_plot[argmin]
        
        plt.scatter([peak[0]],[peak[1]],c=plt.gca().lines[-1].get_color(),s=200)
    plt.legend(ncol=6,loc='lower center')
    plt.ylabel('Elo',weight='semibold')
    locator = mdates.YearLocator()
    # Specify the format - %b gives us Jan, Feb...
    fmt = mdates.DateFormatter('%Y')


    X = plt.gca().xaxis
    X.set_major_locator(locator)
    X.set_major_formatter(fmt)

    if game in ['pk','q4']:
        #plt.xlim(datetime.datetime(2018,9,1),datetime.datetime(2019,1,1))
        locator = mdates.MonthLocator(interval=4)
        # Specify the format - %b gives us Jan, Feb...
        fmt = mdates.DateFormatter('%b%Y')


        X = plt.gca().xaxis
        X.set_major_locator(locator)
        X.set_major_formatter(fmt)
    #if game == 'ql':
    plt.ylim(bottom=1500)
    games_dict = {'unreal':'Unreal Tournament','pk':'Painkiller','ql':"Quake Live",'q2':'Quake 2','q3':'Quake 3','q4': 'Quake 4','qc':'Quake Champions'}
    plt.title('Top 1v1 players in ' +games_dict.get(game,game),weight='medium')
    if game == 'ql':
        plt.xlim(datetime.datetime(2009,6,1), datetime.datetime(2016,10,1))
    plt.tight_layout()
    plt.savefig(game+'.png')

In [ ]:
tt

In [ ]:
rt['cypher']

In [ ]:
df[(df.t1=='cypher') & (df.mtime.map(lambda x: x.year ==2018)) & (df.mtime.map(lambda x: x.month ==8))]

In [ ]:
rt['evil']

In [ ]:
top_teams

In [ ]:
df[(df.game == 'pk') & ((df.t1 == 'ztrider') | (df.t2 == 'ztrider'))  ]

In [ ]:
df[(((df.t1 == 'cypher') & (df.t2 == 'rapha')) | ((df.t1 == 'rapha') & (df.t2 == 'cypher'))) & (df.game == 'ql') ]

In [ ]:
sorted([(v,k) for k,v in ratings.items()],reverse=True)

In [ ]:
def get_top_at(time,team_data,timedel = datetime.timedelta(weeks=30)):
    td = timedel
    rt = team_data
    time = tm
    team_idx = {k: abs(np.array([_[0].replace(tzinfo=None) for _ in v])-tm) for k,v in rt.items() if len(v) > 0}
    team_idx = {k:np.argmin(v) for k,v in team_idx.items() if sum(v < td) > 1}
    top_teams_year = sorted([(round(np.nan_to_num(rt[k][i][1])),k) for k,i in team_idx.items()],reverse=True)
    return top_teams_year

In [ ]:
tm = datetime.datetime(month=6,year=2020,day=8)
top_teams_year = get_top_at(tm,rt)
print(tm,top_teams_year[:13])

In [ ]:
for yr in range(2020,2020):
    tm = datetime.datetime(month=1,year=yr+1,day=1)
    top_teams_year = get_top_at(tm,rt)
    print(tm,top_teams_year[:5])

In [ ]:
_ = plt.hist(df.mtime,100)

In [ ]:
import math

max_year = defaultdict(dict)
for k in rt:
    yrs = set([int(_[0].year) for _ in rt[k]])
    yrs = list(sorted(yrs))
    tm_d = {}
    for y in yrs:
        max_year[y][k] = max([(_[1],_[0]) for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ])

const_W = 1
alt_year = defaultdict(dict)
for k in rt:
    yrs = set([int(_[0].year) for _ in rt[k]])
    yrs = list(sorted(yrs))
    tm_d = {}
    for y in yrs:
        valid_g = [_[1] for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ]
        weights = [const_W] + [1 if valid_g[i] > valid_g[i-1] else const_W for i in range(1,len(valid_g))]
        weights = np.array(weights)
        weights = weights/(np.mean(weights))
        valid_g = np.array(valid_g) * np.array(weights)
        idx = np.argsort(valid_g)#[1:-1]
        #if k == 'czm' and y == 2008:
        #   raise
        valid_g = valid_g[idx]
        weights = weights[idx]
        #num_games  = len([_ for _ in rt[k] if _[0].year == y])**0.15
        if len(valid_g) > 5:
            #alt_year[y][k] =  np.mean(valid_g)
            alt_year[y][k] = ( np.mean([_**2 for _ in valid_g]))**(1/2)
        #alt_year[y][k] = np.sqrt(np.mean([_[1]**2 for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ]))


In [ ]:
rt['unkind']

In [ ]:
#df[((df.t1=='czm') | (df.t2 == 'czm')) & (df.mtime.map(lambda x: x.year == 2008))]

In [ ]:
for yr in range(1995,2021):
    top_teams_year = sorted([(v,k) for k,v in alt_year[yr].items()  if 'sweden' not in k],reverse=True)
    if len(top_teams_year) < 1:
        continue
    print('* **' +str(yr) + '**',end='\t')#,top_teams_year[:5])
    i=0
    sidx = min(5,len(top_teams_year))
    for s,n in top_teams_year[:5]:
        i+=1
        entk = ', ' if i!=sidx else '\n'
        print('{} ({})'.format(n,int(s)),end=entk)

In [ ]:
max_best_ever = []
for yr, res in max_year.items():
    for tm, pelo in res.items():
        if yr != 2000:
            continue
        max_best_ever.append( (pelo[0],pelo[1].strftime("%Y-%m-%d"),str(yr) +'__' + tm))
sorted(max_best_ever,reverse=True)[:50]

In [ ]:
max_best_ever = []
for yr, res in max_year.items():
    for tm, pelo in res.items():
        max_best_ever.append( (pelo[0],pelo[1].strftime("%Y-%m-%d"),str(yr) +'__' + tm))
sorted(max_best_ever,reverse=True)[:50]

In [ ]:
df[df.venue.map(lambda x: 'intel' in x.lower()) & (df.type=='Online')].venue.unique()

In [ ]:
alt_best_ever = []
for yr, res in alt_year.items():
    for tm, pelo in res.items():
        alt_best_ever.append( (pelo,str(yr) +'__' + tm))
sorted(alt_best_ever,reverse=True)[:50]

In [ ]:
df[df.venue == 'Quake Pro League/Season 1/Stage 2/Finals']

In [ ]:
top_teams

In [ ]:
def get_top_year_elo(tm,yr,rt):
    return max([(_[1],_[0]) for _ in rt[tm] if _[0].year == yr])

In [ ]:
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from collections import OrderedDict
import matplotlib.pyplot as plt

In [ ]:
color_dict['av3k'] ='#f15bb5'
color_dict['socrates'] = '#293241'

In [ ]:
plt.figure(figsize=(12,4))
names = ['fatal1ty','zero4','cooller','vo0','toxjq','cypher','rapha']
cmap = plt.get_cmap("tab10")
cmaps = [cmap((i)/float(len(names))) for i in range(len(names))]
years = [range(2000,2008),range(2000,2005),range(2003,2021),range(2004,2006),range(2006,2010),range(2008,2021),range(2009,2021),range(2003,2021),range(2003,2021)]
for n,y,c in zip(names,years,cmaps):
    print(list(y))
    vs = np.array([_ for _ in rt[n] if _[0].year in list(y)])
    c = color_dict[n]
    plt.plot(vs[:,0],vs[:,1],alpha=0.95,label=n,c=c,lw=5)
    for yr in y:
        top_teams_year = sorted([(v,k) for k,v in alt_year[yr].items()],reverse=True)
        for ratingT, tm in top_teams_year[:10]:
            if tm in names:
                continue
            ratings = np.array([_ for _ in rt[tm] if _[0].year == yr])
            plt.plot(ratings[:,0],ratings[:,1],c='k',alpha=0.08,lw=1.5)
plt.legend(loc=4,ncol=7)
plt.ylabel('Elo')
plt.title('Top 1v1 ArenaFPS Players\n(other top players in light gray)')

In [ ]:
df[(df.t1 == 'joker') | (df.t2 == 'joker')]

In [ ]:
dft = df[df.tier == '1']
for n in (dft.t2.value_counts() + dft.t1.value_counts()).dropna().sort_values(0,False).items():
    print(df[((df.t1 == 'cooller') & (df.t2 == n[0])) | ((df.t1 == n[0]) & (df.t2 == 'cooller'))  ].shape[0],n[0])

In [ ]:
df[((df.t1 == 'cooller') & (df.t2 == 'cypher')) | ((df.t1 == 'cypher') & (df.t2 == 'cooller'))  ]

In [ ]:
names = list(df.t1.unique()) + list(df.t2.unique())
names = list(set(names))
alts = []
thing = '?'
for s in sorted(names):
    if thing in s and s.replace(thing,'') in names:
        print(s)
        alts.append((s,s.replace(thing,'')))
print(alts)

In [ ]:
from fuzzywuzzy import process
for s in sorted(names):
    cand= process.extract(s, names, limit=2)[1]
    if cand[1] > 90:
        print((s,cand[0])   )